In [3]:
import cv2

# Load the video
video_path = 'output.mp4'  # Replace with your video path
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    # Get FPS
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"FPS: {fps}")

    # Get total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total Frames: {total_frames}")

    # Calculate video duration (in seconds)
    duration_seconds = total_frames / fps
    print(f"Duration: {duration_seconds} seconds")

    # Optionally, you can release the video capture object if needed
    cap.release()

Error: Could not open video.


Mediapipe observation : 
- Ran on few videos and realised that Mediapipe Pose landmarkers are not able to detect a person if the head is not shown in the frame. Likely because of the top down approach of pose detection.
- Pose landmarkers are trained for closeup cases, so it is difficult to detect a person from far away in the frame. 

In [ ]:
!pip install ultralytics
!pip install opencv-python
!pip install ipywidgets
!pip install matpotlib
!pip install moviepy

In [ ]:
from ultralytics import YOLO

MODEL_NAME = 'yolo11s-pose.pt'

model = YOLO(MODEL_NAME)

# result = model.track( source='video_data/fight_1.mp4', conf=0.7)
result = model.track( source='https://www.youtube.com/watch?v=Sa8gm2F0r8g&ab_channel=SufihanKhanMahabub', conf=0.7)


In [27]:
len(result)

28

In [28]:
import numpy as np
from moviepy import ImageSequenceClip

annotated_frames = []
keypoints_frames = []
for r in result:

    height, width = r.orig_shape

    blank_image = np.zeros((height, width, 3), dtype=np.uint8)

    img = r.plot(img=blank_image, boxes=False)
    keypoints_frames.append(img)

    img = r.plot(boxes=False)
    annotated_frames.append(img)


clip = ImageSequenceClip(keypoints_frames, fps=28/20)  # Adjust fps as needed
clip.write_gif("keypoints.gif", fps=28/20)

clip = ImageSequenceClip(annotated_frames, fps=28/20)  # Adjust fps as needed
clip.write_gif("annotated.gif", fps=28/20)


MoviePy - Building file keypoints.gif with imageio.


MoviePy - Building file annotated.gif with imageio.


In [29]:
import cv2
import numpy as np
import torch

# Assuming you have a list of frames, each containing keypoint data

# Function to draw keypoints on an image
def draw_keypoints(image, xy, xyn=None, conf=None):
    for point in xy:
        cv2.circle(image, (int(point[0]), int(point[1])), 5, (0, 0, 255), -1)
    return image

# Function to join frames into a GIF
def create_gif(frames, output_path='output.gif'):
    # Create a VideoWriter object to save the GIF
    height, width = frames[0].keypoints.data.cpu().shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'gif ')
    out = cv2.VideoWriter(output_path, fourcc, 1.0, (width, height))

    for frame in frames:
        img = np.zeros((height, width, 3), dtype=np.uint8)
        keypoints_data = frame.keypoints.data.cpu().permute(1, 2, 0).numpy() * 255
        if keypoints_data.shape != (height, width, frame.keypoints.data.cpu().shape[0]):
            raise ValueError("Keypoint data dimensions do not match image dimensions")
        
        img[:height, :width] = keypoints_data
        img = draw_keypoints(img.clone(), frame['xy'])
        out.write(img)

    out.release()

# Example usage
create_gif(result, 'keypoint_animation.gif')

[ WARN:0@6367.409] global cap.cpp:779 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.11.0) /io/opencv/modules/videoio/src/cap_images.cpp:415: error: (-215:Assertion failed) !filename_pattern.empty() in function 'CvVideoWriter_Images'




ValueError: Keypoint data dimensions do not match image dimensions

In [ ]:
import pandas as pd

data = []
# Assuming 'results' is from your prediction
for frame_id, result in enumerate(results):
    for person_id, person in enumerate(result.keypoints):
        keypoints = person.xy  # Get keypoints
        confidences = person.conf  # Get confidences
        for kp_id, (xy, conf) in enumerate(zip(keypoints, confidences)):
            label = labels[kp_id]  # Get the label from predefined labels list
            data.append([frame_id, person_id, label, *xy, conf])

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data, columns=['frame_id', 'person_id', 'label', 'x', 'y', 'confidence'])
